# Lyft & Uber Rideshare Data Analysis
### By Nadya  Pena

The following is an example of routine outlier or anomaly detection in data using simple data analytics and statistics. 
The dataset we will be using is a sample of Uber and Lyft rideshare information.

The first few col schema of the data are as follows:


|field|type   |
| --- | --- |
|merchant_name|string |
|order_number|string |
|user_id| string|
|order_time|timestamp|
|email_time|timestamp|
|insert_time|timestamp|
|update_time|timestamp|
|order_total_amount|double|
|order_points|string|
|order_shipping|string|
|order_tax|double|
|order_subtotal|double|
.
.
.

Steps of analysis:
1. We'll start by loading our data (whether from a table or a file) and we'll read it into a Spark Dataframe.
2. Then we'll do some high-level data quality checks and note things like missing values, duplicate records, value distributions, and anything else remarkable.
3. Once we've done that, we can look closer at individual fields to see what their values are like. 
4. Finally, do more specific analytics like looking at average ride counts and ride length per weekday. Use this average to compare ride counts and ride lengths. We can use Z-scores to determine if particular rides or ride counts are unusual. 

### Imports and Spark session creation

In [1]:
from pyspark.sql import SparkSession, SQLContext, HiveContext
from pyspark.sql.types import *
from pyspark import SparkConf, SparkContext
import datetime
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.window import Window
import pandas as pd

In [ ]:
sc = SparkSession.builder.master("local").appName("RideShare").getOrCreate()

In [ ]:
spark = SparkSession(sc)
hiveContext = HiveContext(sc)
SQLContext = SQLContext(sc)

### Read in the Data 
In this case we'll be reading from a CSV but if you're not reading from a CSV you can read from a DB as well using a spark-based driver for whatever db you're using.

In [ ]:
df = spark.read.format('csv') \
        .options(header='true', inferSchema='true') \
        .load(r"C:\Users\nadya\Documents\RideShareProject\Data_Rideshare.csv")

In [ ]:
df.printSchema()

In [ ]:
df.columns

In [ ]:
df.take(2)

## High-Level Data Quality Checks
The following few cells are simply checking for:
1. Missing Data
2. Duplicate Records
3. Number of Records add up

In [ ]:
# ascertain the number of distinct merchants
distinct_merchants = df.select(df.merchant_name).distinct()
distinct_merchants.show()

In [ ]:
# Count all records for each service
print("Total number of records :", df.count())
print("Total Lyft records: ", df.filter(df.merchant_name == 'Lyft').count())
print("Total Uber records: ", df.filter(df.merchant_name == 'Uber').count())

In [ ]:
# create a temp table from the Data Frame.
df.registerTempTable("Rideshares")
result = hiveContext.sql("SELECT COUNT(*) AS records FROM Rideshares").collect()

In [ ]:
result

### Data Redundancy check (duplicate records)
If duplicate records exist, ascertain that they are valid

In [ ]:
# count distinct records 
df.distinct().count()

### Data Missingness Check
1. Get the number of records with null values. Investigate nulls to see if they are valid.

2. Calculate proportion of missing data per field. 
    * Fields with high proportion of missing values may not be useful for analysis. 
    * Could be potential anomalies if a field that is usually populated turns up blank

In [ ]:
# Count number of records that have at least 1 missing field

query = "SELECT * \
         FROM Rideshares \
         WHERE merchant_name IS NULL OR \
               user_id IS NULL OR \
               order_number IS NULL OR \
               order_time IS NULL OR \
               email_time IS NULL OR \
               insert_time IS NULL OR \
               update_time IS NULL OR \
               order_total_amount IS NULL OR \
               order_points IS NULL OR \
               order_shipping IS NULL OR \
               order_tax IS NULL OR \
               order_subtotal IS NULL OR \
               order_total_qty IS NULL OR \
               product_description IS NULL OR \
               product_subtitle IS NULL OR \
               item_quantity IS NULL OR \
               item_price IS NULL OR \
               digital_transaction IS NULL OR \
               checksum IS NULL OR \
               product_reseller IS NULL OR \
               product_category IS NULL OR \
               order_discount IS NULL OR \
               SKU IS NULL OR \
               item_id IS NULL OR \
               order_pickup IS NULL OR \
               from_domain IS NULL OR \
               email_subject IS NULL OR \
               delivery_date IS NULL OR \
               start_source_folder_date IS NULL OR \
               end_source_folder_date IS NULL OR \
               file_id IS NULL OR \
               source_dttimestamp IS NULL OR \
               dttimestamp IS NULL"
result = hiveContext.sql(query)
result.count()

In [ ]:
# Calculate proportion of missingness for each column
# columns with high % of NULLS may not be useful for analysis
count = df.count()
print(count)
for col in df.columns:
    query = "SELECT \
            CAST( \
                SUM( \
                    CASE WHEN {0} IS NULL THEN 1 ELSE 0 END) \
             AS DECIMAL(10,2))/ COUNT(*) AS missing_{1}, \
             SUM(CASE WHEN {0} IS NULL THEN 1 ELSE 0 END) AS how_many_null \
            FROM Rideshares".format(col, col)
    result = hiveContext.sql(query)
    result.show()
    

In [ ]:
# look records where order_time is null. Are those valid? perhaps. 
# Seems these could be other activities related to tipping, corrections, and payment. 
query= "SELECT \
        user_id, \
        date(order_time), \
        email_subject, \
        order_total_amount \
        FROM Rideshares \
        WHERE date(order_time) IS NULL \
        ORDER BY date(order_time)"
result = hiveContext.sql(query)
result.show()

In [ ]:
query= "SELECT \
        merchant_name, \
        email_subject, \
        order_total_amount \
        FROM Rideshares \
        WHERE date(order_time) IS NOT NULL AND merchant_name = 'Uber'\
        ORDER BY date(order_time)"
result = hiveContext.sql(query)
result.show()

### Data Values Exploration
Try to understand the expected "look" of the data
> (i.e. do some fields always have same value, or should the field look different for all records?)

In [ ]:
# Get list of distinct values for fields. 
# If the field has same value for all records, it might not be very useful for anomaly detection. 
# Ex: the field item_quantity and order_total_qty are the same for all records
fields = [ 'item_quantity', 'order_total_qty',]
for col in fields:
    query = "SELECT \
            DISTINCT({0}) \
            FROM Rideshares".format(col)
    result = hiveContext.sql(query)
    result.show()

#### Quick Evaluation of Time fields
* Time related queries to determine when each time field starts and ends</dt>
* note difference between time fields to see how they relate to eachother</dt>
* Look at time series trends later on

In [ ]:
# Determine the starting date for each of the date fields
cols = [ 'source_dttimestamp','dttimestamp', 'order_time','email_time','insert_time','update_time',]
for col in cols:
    query = "SELECT \
                 MIN({0}), \
                 MAX({0}) \
                 FROM Rideshares".format(col)
    result = hiveContext.sql(query)
    result.show()

In [ ]:
# Determine the starting date for each of the date fields
cols = [ 'source_dttimestamp','dttimestamp', 'order_time','email_time','insert_time','update_time',]

for col in cols:
    query = "SELECT \
             date({0}) AS {0}_col, \
             COUNT(*) AS counts\
             FROM Rideshares\
             GROUP BY {0}_col \
             ORDER BY {0}_col ASC".format(col)
    result = hiveContext.sql(query)
    result.show()

### Z-scores
> Z-scores are used to measure an observation's deviation from the group's mean value.
Z-scores reveal whether a score is typical for a specified data set or if it is atypical

In the below cells we look at two examples of how z-scores can help us catch unusual values.
* The first is screening for unusually expensive rides by looking at the **order_total_amount** field.
* The second is screening data for unusual ride count based on avg weekly ride count

In [ ]:
# Get the order_total_amount for each order and rank them from greatest
# Taking the z-score of each order_total_amount can tell us if the value is unusual
# From this data we see that there are some rides with unusually high prices that
# deviate from the overall avg ride total
# NOTE: the avg was not normalized by merchant name so the avg is across all rides regardless of merchant
query = "SELECT merchant_name, \
        order_number, \
        user_id, \
        item_price, \
        order_total_amount, \
        (order_total_amount - AVG(order_total_amount) over()) /stddev(order_total_amount) over() AS ota_zscore \
        FROM Rideshares \
        WHERE order_total_amount IS NOT NULL \
        ORDER BY ota_zscore DESC"
result = hiveContext.sql(query)
result.show()

In [ ]:
# get number of records with unusual z-scores for order_total_amount, grouped by service
# We see that Uber has many more order_total_amounts with z-score above 4 or below -4 
query = "SELECT \
            B.merchant_name, \
            COUNT(B.ota_zscore) \
         FROM \
             (SELECT \
                 merchant_name, \
                 (order_total_amount - AVG(order_total_amount) over()) /stddev(order_total_amount) over() AS ota_zscore\
              FROM Rideshares)B \
         WHERE ota_zscore > 4 OR ota_zscore < -4 \
         GROUP BY merchant_name"

result = hiveContext.sql(query)
result.show()
              

### Look for Unusual Values in Ride Volume 
> In this next example we will look at daily rides and compare them to average rides for that given weekday
> Ex: we will compare a Monday ride count to average Monday ride count and see if the ride count is unusual for a Monday
 * data is taken separately for lyft and uber so lyft rides will be compared against lyft averages
 * Likewise, Uber rides will be compared to Uber averages

In [ ]:
query = "SELECT \
            B.merchant_name, \
            B.weekday, \
            AVG(B.ride_counts) AS avg_rides\
            FROM( \
             SELECT \
                 date(order_time), \
                 (CASE WHEN DAYOFWEEK(date(order_time)) = 1 THEN 'SUNDAY' \
                       WHEN DAYOFWEEK(date(order_time)) = 2 THEN 'MONDAY' \
                       WHEN DAYOFWEEK(date(order_time)) = 3 THEN 'TUESDAY' \
                       WHEN DAYOFWEEK(date(order_time)) = 4 THEN 'WEDNESDAY' \
                       WHEN DAYOFWEEK(date(order_time)) = 5 THEN 'THURSDAY' \
                       WHEN DAYOFWEEK(date(order_time)) = 6 THEN 'FRIDAY' \
                       WHEN DAYOFWEEK(date(order_time)) = 7 THEN 'SATURDAY' \
                  END) weekday, \
                 merchant_name, \
                 COUNT(*) AS ride_counts \
             FROM Rideshares \
             WHERE date(order_time) IS NOT NULL \
             GROUP BY \
                date(order_time), \
                merchant_name \
            ) B \
        GROUP BY B.merchant_name, B.weekday \
        ORDER BY merchant_name, avg_rides DESC"
rides_per_weekday_result = hiveContext.sql(query)
rides_per_weekday_result.show()

In [ ]:
# Register a new temp table from dataframe
rides_per_weekday_result.registerTempTable("Rideshares_per_weekday")

In [ ]:
#rides_per_weekday_result.filter(rides_per_weekday_result.weekday == 'SATURDAY').show()

In [ ]:
# plot how many riders there were for each service per day.
# an active user is someone who makes at least 2 transactions per quarter
query = "SELECT \
         date(order_time), \
         (CASE WHEN DAYOFWEEK(date(order_time)) = 1 THEN 'SUNDAY' \
               WHEN DAYOFWEEK(date(order_time)) = 2 THEN 'MONDAY' \
               WHEN DAYOFWEEK(date(order_time)) = 3 THEN 'TUESDAY' \
               WHEN DAYOFWEEK(date(order_time)) = 4 THEN 'WEDNESDAY' \
               WHEN DAYOFWEEK(date(order_time)) = 5 THEN 'THURSDAY' \
               WHEN DAYOFWEEK(date(order_time)) = 6 THEN 'FRIDAY' \
               WHEN DAYOFWEEK(date(order_time)) = 7 THEN 'SATURDAY' \
          END) weekday, \
         merchant_name, \
         COUNT(*) AS ride_counts \
         FROM Rideshares \
         WHERE date(order_time) IS NOT NULL \
         GROUP BY date(order_time), merchant_name \
         ORDER BY date(order_time)"
result = hiveContext.sql(query)
result.show()

In [ ]:
query = " SELECT \
            B.ride_date, \
            B.weekday, \
            B.merchant_name, \
            B.ride_counts AS actual_ride_count, \
            CAST(C.avg_rides AS INT), \
            CAST((B.ride_counts - C.avg_rides) AS INT) AS rides_diff, \
            (B.ride_counts - C.avg_rides)/ stddev(B.ride_counts) over() AS zscore \
            FROM( \
                SELECT \
                 date(order_time) AS ride_date, \
                 (CASE WHEN DAYOFWEEK(date(order_time)) = 1 THEN 'SUNDAY' \
                       WHEN DAYOFWEEK(date(order_time)) = 2 THEN 'MONDAY' \
                       WHEN DAYOFWEEK(date(order_time)) = 3 THEN 'TUESDAY' \
                       WHEN DAYOFWEEK(date(order_time)) = 4 THEN 'WEDNESDAY' \
                       WHEN DAYOFWEEK(date(order_time)) = 5 THEN 'THURSDAY' \
                       WHEN DAYOFWEEK(date(order_time)) = 6 THEN 'FRIDAY' \
                       WHEN DAYOFWEEK(date(order_time)) = 7 THEN 'SATURDAY' \
                  END) weekday, \
                 merchant_name, \
                 COUNT(*) AS ride_counts \
                 FROM Rideshares \
                 WHERE date(order_time) IS NOT NULL \
                 GROUP BY date(order_time), merchant_name \
                ) B \
             LEFT JOIN Rideshares_per_Weekday AS C \
                 ON B.weekday = C.weekday AND B.merchant_name = C.merchant_name \
            ORDER BY zscore"
result = hiveContext.sql(query)
result.show()

### As seen from the table above...
There are ride counts that deviate from the mean ride count by several std deviations. For example the first few rows show significantly fewer Uber rides than the usual for a Saturday and Friday. Likewise if we were to order by decreasing z-scores, we might see a surge in rides that is unusual as well. Depending on our tolerance for deviations, these may or may not be outliers. 
#### One suggestion for use of this data is to look at instances where there is complimentary deviation in Lyft and Uber ride counts. 
> For example, if Uber rides go down at the same time as Lyft rides go up, this could indicate customer defection from one platform to the other. It would be especially interesting if Uber rides go down and lyft goes up by a similar amount to Uber's lost rides. 